## Installs
_Do it only once_

In [1]:
# based on azureml_py310_sdkv2 kernel
# %pip install torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0
# %pip install 'monai[nibabel, ignite, tqdm]'
# %pip install itkwidgets
# %pip install --upgrade azure-ai-ml #to have SDK 2 (1.5.0)

## Import Libraries

In [5]:
from azure.ai.ml import MLClient, Input
from azure.identity import DefaultAzureCredential

## Define global variables

In [6]:
# Training
experiment_name = 'monai-brain-tumor-from-registry' 
train_target = 'Standard-NC96ads-A100-v4'

# Registry
registry_name="ams-components"
registry_location="westeurope"

#Registry Components
upload_data_from_blob_name="upload_data_from_blob"
train_segmentation_name="train_segmentation"
monai_pipeline_name="monai_pipeline"
tar_data_asset_local_name="tar_data_asset_local"

## Create Azure ML and Registry Clients

In [7]:
credential = DefaultAzureCredential()

ml_client = MLClient.from_config(credential=credential)

ml_client_registry = MLClient(credential=credential,
                        registry_name=registry_name,
                        registry_location=registry_location)

Found the config file in: ./config.json


## Upload Refistry components

In [9]:
# Upload data
tar_data_from_registry = ml_client_registry.data.get(name=tar_data_asset_local_name, label='latest')
# Upload Command components
upload_component = ml_client_registry.components.get(name=upload_data_from_blob_name, label='latest')
train_component = ml_client_registry.components.get(name=train_segmentation_name, label='latest')
#Upload pipeline
pipeline_from_registry = ml_client_registry.components.get(name=monai_pipeline_name, label='latest')

## Run Pipeline Job

In [10]:
pipeline_job = pipeline_from_registry(
       pipeline_input_file=Input(type="uri_file", path=tar_data_from_registry.id)
)

pipeline_job.settings.default_compute = train_target


In [11]:
ml_client.jobs.create_or_update(pipeline_job, experiment_name=experiment_name)

Experiment,Name,Type,Status,Details Page
monai-brain-tumor-from-registry,careful_street_3r8v68fj02,pipeline,Preparing,Link to Azure Machine Learning studio
